In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 60.1 MB/s 
     |████████████████████████████████| 86 kB 4.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [76]:
!pip install haversine

In [445]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import warnings
from PIL import Image
import re
from haversine import haversine
warnings.filterwarnings('ignore')
%matplotlib inline

In [317]:
df = pd.read_csv('program_keyword.csv', encoding = 'cp949')
df.head()

,시설명,위도,경도,주소,전화번호,과정구분,과정명,인원,요일,시간,키워드리스트
0,구립 곰달래어르신복지센터,37.529057,126.839750,강서구 강서로5길50 (화곡동),2699-8264,교육,세계사,9,금,10:00~10:50,"교육,역사,세계사,세계,공부"
1,시립 강서어르신종합복지관,37.558089,126.838555,강서구 화곡로61길85(등촌동),3664-0322~4,교육,인권교육(반딧불),센터문의바람,화,10:00~12:00,"교육,인권,반딧불,사람,권리,인권교육,공부"
2,시립 강서어르신종합복지관,37.558089,126.838555,강서구 화곡로61길85(등촌동),3664-0322~4,교육,빨간모자 스토리(이야기꾼 양성교육),센터문의바람,금,10:00~12:00,"교육,이야기,이야기꾼,빨간모자,스토리,빨간모자스토리,공부"
3,시립 강서어르신종합복지관,37.558089,126.838555,강서구 화곡로61길85(등촌동),3664-0322~4,교육,슬기로운 노인생활 (생활정보),온라인-자율,금,센터문의바람,"교육,슬기로운,생활정보,생활,정보,노인,생활,공부"
4,구립 곰달래어르신복지센터,37.529057,126.839750,강서구 강서로5길50 (화곡동),2699-8264,디지털,컴퓨터 기초,5,월~금,10:00~11:50,"디지털,디지털교육,컴퓨터"


In [318]:
gang = pd.read_csv('강서구마지막.csv', encoding = 'cp949')
gang.head()

,공원이름,위도,경도
0,모태,37.529458,126.849515
1,까치,37.533388,126.841895
2,새나라,37.561528,126.852025
3,다운,37.532059,126.844653
4,배다리,37.532073,126.848303


In [319]:
df = df.dropna()

In [320]:
df['키워드리스트'] = df['키워드리스트'].apply(lambda x : x.split(','))

In [414]:
df

,시설명,위도,경도,주소,전화번호,과정구분,과정명,인원,요일,시간,키워드리스트
0,구립 곰달래어르신복지센터,37.529057,126.839750,강서구 강서로5길50 (화곡동),2699-8264,교육,세계사,9,금,10:00~10:50,"[교육, 역사, 세계사, 세계, 공부]"
1,시립 강서어르신종합복지관,37.558089,126.838555,강서구 화곡로61길85(등촌동),3664-0322~4,교육,인권교육(반딧불),센터문의바람,화,10:00~12:00,"[교육, 인권, 반딧불, 사람, 권리, 인권교육, 공부]"
2,시립 강서어르신종합복지관,37.558089,126.838555,강서구 화곡로61길85(등촌동),3664-0322~4,교육,빨간모자 스토리(이야기꾼 양성교육),센터문의바람,금,10:00~12:00,"[교육, 이야기, 이야기꾼, 빨간모자, 스토리, 빨간모자스토리, 공부]"
3,시립 강서어르신종합복지관,37.558089,126.838555,강서구 화곡로61길85(등촌동),3664-0322~4,교육,슬기로운 노인생활 (생활정보),온라인-자율,금,센터문의바람,"[교육, 슬기로운, 생활정보, 생활, 정보, 노인, 생활, 공부]"
4,구립 곰달래어르신복지센터,37.529057,126.839750,강서구 강서로5길50 (화곡동),2699-8264,디지털,컴퓨터 기초,5,월~금,10:00~11:50,"[디지털, 디지털교육, 컴퓨터]"
...,...,...,...,...,...,...,...,...,...,...,...
154,구립 화곡어르신복지센터,37.535453,126.832312,강서구 월정로30길96 (화곡동),2605-6900,체육,기필라테스,15,금,16:10~17:10,"[체육, 운동, 건강, 춤, 댄스, 라인댄스, 라인]"
155,시립 강서어르신종합복지관,37.558089,126.838555,강서구 화곡로61길85(등촌동),3664-0322~4,체육,영상수업(체조),온라인-자율,수,유튜브,"[체육, 운동, 건강, 춤, 댄스]"
156,시립 강서어르신종합복지관,37.558089,126.838555,강서구 화곡로61길85(등촌동),3664-0322~4,체육,슬기로운 집콕생활 (체조),온라인-자율,목,센터문의바람,"[체육, 운동, 건강, 춤, 댄스]"
157,시립 강서어르신종합복지관,37.558089,126.838555,강서구 화곡로61길85(등촌동),3664-0322~4,체육,라이브방송 - 선배시민아카데미,온라인-자율,목,10:30~11:30,"[체육, 운동, 건강, 춤, 댄스, 줌바, 줌바댄스]"


In [323]:
from gensim.models.word2vec import Word2Vec

def model(keyword) :
    model=Word2Vec(df['키워드리스트'],sg=1, window=15, min_count=1)
    model.init_sims(replace=True)
    result = model.wv.most_similar(keyword)
    return result

In [ ]:
def distance(df1, df2):
    df_new = []
    for i in range(len(df2)):
        start = (float(df1['위도'].values), float(df1['경도'].values))
        end = (float(df2['위도'][i]), float(df2['위도'][i]))
        dist = haversine(start, end, unit='m')
        df_new.append(dist)
    return(df_new)

In [430]:
def recommend(word, park) : 
  target = model(word)

  keyword = []
  for i in target[:1] : 
    keyword.append(i[0])
    keyword.append(word)

  indexes = []
  for i in range(len(df)) : 
      for j in range(len(df.loc[i, '키워드리스트'])) : 
          if df['키워드리스트'][i][j] in keyword : 
              indexes.append(i)
              break

  후보 = df.iloc[indexes,]

  후보시설  = 후보['시설명'].unique()
  후보위도 = 후보['위도'].unique()
  후보경도 = 후보['경도'].unique()

  data = pd.DataFrame({'시설명' : list(후보시설), '위도' : list(후보위도), '경도' : list(후보경도)})

  dist = distance(gang[gang['공원이름 ']==park], data)

  복지관 = data.iloc[dist.index(min(dist))]['시설명']

  add_num = df[df['시설명']==복지관][['주소','전화번호', '위도', '경도']]
  address = add_num.iloc[0,0]
  phone = add_num.iloc[0,1]
  lat = add_num.iloc[0,2]
  lng = add_num.iloc[0,3]

  programs = 후보[후보['시설명'] == 복지관]['과정명']

  return 복지관, address, phone, programs, lat, lng

In [431]:
복지관, 주소, 전화번호, 프로그램, 위도, 경도 = recommend('디지털', '모태')

In [467]:
import folium

a = folium.Map(location = [37.5580891, 126.8450994], zoom_start = 17)
folium.Marker([위도, 경도], popup='The Waterfront').add_to(a)
folium.CircleMarker([위도, 경도], radius=50, popup = 'Laurelasdf', color = '#3186cc', fill_color = '#3186cc').add_to(a)

In [465]:
print('인근 복지관 이름 : ', 복지관)
print('복지관 주소 : ', 주소)
print('복지관 전화번호 : ', 전화번호)
print()
a

인근 복지관 이름 :  시립 강서어르신종합복지관
복지관 주소 :  강서구 화곡로61길85(등촌동)
복지관 전화번호 :  3664-0322~4



In [469]:
print('[추천 프로그램]')
print()
for i in 프로그램 :
  print(i)

[추천 프로그램]

인권교육(반딧불)
빨간모자 스토리(이야기꾼 양성교육)
슬기로운 노인생활 (생활정보)
스마트폰1
스마트폰2
디지털컨설턴트
컴퓨터기초반1
컴퓨터기초반2
인터넷기초반1
인터넷기초반2
인터넷검색반1
인터넷검색반2
인터넷카페반1
인터넷카페반2
zoom 교육반
문서작성반
